In [1]:
import sys, win32com.client
import sapscript as ss
from IPython.display import display

In [2]:
sap = ss.sap()

In [3]:
try:
    sap.check_system('ERT')
    session = sap.session()
    
    sap.run('zimkr')
    session.findById("wnd[0]/usr/ctxtS_WRK-LOW").text = "2024"
    session.findById("wnd[0]/tbar[1]/btn[8]").press()
    cols = ['VBELN', 'ADDTI', 'CPROC', 'INITS_GDS', 'DDLTI']
    cols_name = ['Заказ', 'ВремяДобавл', 'ОБраб','ИсхСтоим', 'СобратьК']
    grid = session.findById("wnd[0]/usr/cntlALV_0100/shellcont/shellcont/shell/shellcont[0]/shell")
    table = sap.read_alv(grid, cols, cols_name)
    display(table)
except:
    sap.error_print()
finally:
    session = None

,Заказ,ВремяДобавл,ОБраб,ИсхСтоим,СобратьК
0,5630054763,29.08.19 17:39:06,,"2 598,00",30.08.19 12:00:00
1,5630054762,29.08.19 17:39:01,X,"2 598,00",30.08.19 12:00:00
2,5630054761,29.08.19 17:28:38,X,"2 598,00",30.08.19 12:00:00
3,5630054760,29.08.19 17:28:33,X,"2 598,00",30.08.19 12:00:00
4,5630054759,29.08.19 17:28:21,X,"2 598,00",30.08.19 12:00:00
5,5632000685,29.08.19 16:39:49,,"0,00",29.08.19 17:39:49
6,5632000684,29.08.19 16:37:52,X,"0,00",29.08.19 17:37:52
7,5632000683,29.08.19 15:52:30,,"59,00",29.08.19 18:00:00
8,5630054709,29.08.19 15:43:24,X,"2 598,00",30.08.19 12:00:00
9,5630054708,29.08.19 15:43:18,,"2 598,00",30.08.19 12:00:00


In [24]:
table

,Заказ,ВремяДобавл,ОБраб,ИсхСтоим,СобратьК
0,5632000683,29.08.19 15:52:30,,"59,00",29.08.19 18:00:00
1,5630054709,29.08.19 15:43:24,,"2 598,00",30.08.19 12:00:00
2,5630054708,29.08.19 15:43:18,,"2 598,00",30.08.19 12:00:00
3,5630054707,29.08.19 15:43:04,,"2 598,00",30.08.19 12:00:00
4,5632000678,29.08.19 15:36:20,,"672,00",29.08.19 16:36:20
5,5632000677,29.08.19 15:30:42,,"279,89",29.08.19 16:30:42
6,5630054702,29.08.19 14:49:58,X,"2 598,00",30.08.19 12:00:00
7,5630054701,29.08.19 14:31:16,,"2 598,00",29.08.19 15:31:16
8,5630054700,29.08.19 14:31:00,X,"2 598,00",30.08.19 12:00:00
9,5632000675,29.08.19 14:19:47,,"325,00",29.08.19 15:19:47
